In [6]:
import sympy as sp
from IPython.display import display, Latex
import pandas as pd

In [26]:
# Definición de las variables
x1, x2 = sp.symbols('x1 x2')
#x1,x2,x3,x4,x5 = sp.symbols('x1 x2 x3 x4 x5') #Ejercicio 4
x = sp.Matrix([x1, x2])
#x = sp.Matrix([x1, x2, x3, x4, x5])
display(Latex('Defino variable $x$'))
display(Latex('$x = ' + sp.latex(x) + '$'))

# Definición de la función objetivo
#f = x1**2 + x2**2 #Base
#f = x1**2 + 0.5*x2**2 #Ejercicio 1
#f = 3*x1**2 + x2**4 #Ejercicio 2
f = (x1-2)**2 + (x1-2*x2)**2 #Ejercicio 3
#f = (x1)**2 +(x2)**2 +(x3)**2 +(x4)**2 +(x5)**2 #Ejercicio 4
display(Latex('Función objetivo $f(x)$'))
display(Latex('$f(x) = ' + sp.latex(f) + '$'))

# Punto inicial
#x0 = sp.Matrix([1,1]) #Base
#x0 = sp.Matrix([2,1]) #Ejercicio 1
#x0 = sp.Matrix([1,-2]) #Ejercicio 2
x0 = sp.Matrix([0,3]) #Ejercicio 3
#x0 = sp.Matrix([1,-2,3,-4,5]) #Ejercicio 4
display(Latex('Punto inicial $x_0$'))
display(Latex('$x_0 = ' + sp.latex(x0) + '$'))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [27]:
# Calculamos el gradiente de f
grad_f = sp.Matrix([sp.diff(f, xi) for xi in x])
display(Latex(f'Gradiente de $f(x)$: $\\nabla f = {sp.latex(grad_f)}$'))

<IPython.core.display.Latex object>

In [28]:
# Función para la búsqueda de Armijo
def armijo_line_search(xk, dk, alpha=1, beta=0.5, sigma=0.5):
    m_k = 0
    while True:
        x_new = xk + alpha * beta**m_k * dk
        lhs = f.subs({x[i]: x_new[i] for i in range(len(x))})
        rhs = f.subs({x[i]: xk[i] for i in range(len(x))}) + sigma * alpha * beta**m_k * -(grad_f.subs({x[i]: xk[i] for i in range(len(x))}).norm())**2
        if lhs <= rhs:
            break
        m_k += 1
    return alpha * beta**m_k, m_k

In [29]:
# Función para la búsqueda unidimensional exacta
def exact_line_search(xk, dk):
    lambda_ = sp.symbols('lambda')
    g = f.subs({x[i]: xk[i] + lambda_ * dk[i] for i in range(len(x))})
    g_prime = sp.diff(g, lambda_)
    lambda_solutions = sp.solve(g_prime, lambda_)
    
    # Filtrar solo soluciones reales
    lambda_real = [sol.evalf() for sol in lambda_solutions if sol.is_real]
    
    # Si no hay soluciones reales, usar un valor predeterminado
    if not lambda_real:
        return 1  # Puede ajustarse según sea necesario
    
    return min(lambda_real, key=abs)

In [30]:
# Método de descenso de gradiente
def gradient_descent(f, grad_f, x0, method='exact', epsilon=1e-5, max_iter=1000):
    xk = x0
    iteraciones = []
    k = 0
    iter_inner = 0
    while True:
        grad_fk = grad_f.subs({x[i]: xk[i] for i in range(len(x))})
        norm_grad_fk = grad_fk.norm()
        display(Latex(f'$||\\nabla f(x_k)|| = {norm_grad_fk.evalf()}$'))
        if norm_grad_fk < epsilon or k >= max_iter:
            break
        dk = -grad_fk
        if method == 'exact':
            lambda_k = exact_line_search(xk, dk)
            iter_inner = 1  # Exact line search is counted as one iteration
        elif method == 'armijo':
            lambda_k, iter_inner = armijo_line_search(xk, dk)
        xk = xk + lambda_k * dk
        iteraciones.append((k, iter_inner, xk, f.subs({x[i]: xk[i] for i in range(len(x))}), grad_fk, dk, lambda_k))
        display(Latex(f'Iteración {sp.latex(k)}: $x_k = {sp.latex(xk)}, \\nabla f(x_k) = {sp.latex(tuple(grad_fk))}$'))
        display(Latex(f'Actualización: $x_{{k+1}} = {sp.latex(xk)}, \\lambda = {lambda_k}$'))
        k += 1
    return xk, iteraciones

In [31]:
# Optimización usando búsqueda unidimensional exacta
x_opt_exact, iteraciones_exact = gradient_descent(f, grad_f, x0, method='exact')

# Mostrar resultado final para búsqueda unidimensional exacta
display(Latex("**Resultados usando Búsqueda Unidimensional Exacta**"))
display(Latex(f"El punto óptimo es: ${sp.latex(x_opt_exact)}$"))
display(Latex(f"Valor óptimo de la función: ${sp.latex(f.subs({x[i]: x_opt_exact[i] for i in range(len(x))}))}$"))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [32]:
# Mostrar iteraciones para búsqueda unidimensional exacta
for i, (k, it ,xk, fk, grad_fk, dk, tk) in enumerate(iteraciones_exact):
    display(Latex(f"\nIteración {i+1}:"))
    display(Latex(f"$x_k = {sp.latex(xk)}$"))
    display(Latex(f"$f(x_k) = {sp.latex(fk)}$"))
    display(Latex(f"$\\nabla f(x_k) = {sp.latex(grad_fk)}$"))
    display(Latex(f"$d_k = {sp.latex(dk)}$"))
    display(Latex(f"$\lambda_k = {tk}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [33]:
# Optimización usando búsqueda de Armijo
x_opt_armijo, iteraciones_armijo = gradient_descent(f, grad_f, x0, method='armijo')

# Mostrar resultado final para búsqueda de Armijo
display(Latex("**Resultados usando Búsqueda de Armijo**"))
display(Latex(f"El punto óptimo es: ${sp.latex(x_opt_armijo)}$"))
display(Latex(f"Valor óptimo de la función: ${sp.latex(f.subs({x[i]: x_opt_armijo[i] for i in range(len(x))}))}$"))


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [34]:
# Mostrar iteraciones para búsqueda de Armijo
for i, (k,it,  xk, fk, grad_fk, dk, tk) in enumerate(iteraciones_armijo):
    display(Latex(f"\nIteración {i+1}:"))
    display(Latex(f"$x_k = {sp.latex(xk)}$"))
    display(Latex(f"$f(x_k) = {sp.latex(fk)}$"))
    display(Latex(f"$\\nabla f(x_k) = {sp.latex(grad_fk)}$"))
    display(Latex(f"$d_k = {sp.latex(dk)}$"))
    display(Latex(f"$\lambda_k = {tk}$"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [35]:
# Preparar la tabla comparativa
def preparar_datos(iteraciones, metodo):
    datos = []
    for iteracion in iteraciones:
        k, iter_inner, xk, fk, grad_fk, dk, lambda_k = iteracion
        datos.append([metodo, k, iter_inner, tuple(xk.evalf()), fk.evalf(), tuple(grad_fk.evalf()), grad_fk.norm().evalf(), lambda_k])
    return datos

datos_exact = preparar_datos(iteraciones_exact, 'Exacto')
datos_armijo = preparar_datos(iteraciones_armijo, 'Armijo')

# Crear un DataFrame para cada método
columnas = ['Método', 'Iteración General', 'Iteración Interna', 'x_k', 'f(x_k)', 'grad_f(x_k)', '||grad_f(x_k)||', 'lambda_k']
df_exact = pd.DataFrame(datos_exact, columns=columnas)
df_armijo = pd.DataFrame(datos_armijo, columns=columnas)

# Mostrar las tablas comparativas
display(Latex("**Resultados usando Búsqueda Unidimensional Exacta**"))
display(df_exact)
display(Latex("**Resultados usando Búsqueda de Armijo**"))
display(df_armijo)

<IPython.core.display.Latex object>

,Método,Iteración General,Iteración Interna,x_k,f(x_k),grad_f(x_k),||grad_f(x_k)||,lambda_k
0,Exacto,0,1,"(1.52941176470588, 0.705882352941177)",0.235294117647059,"(-16.0000000000000, 24.0000000000000)",28.8444102037119,0.0955882352941176
1,Exacto,1,1,"(1.98823529411765, 1.01176470588235)",0.00138408304498250,"(-0.705882352941178, -0.470588235294116)",0.848365005991527,0.649999999999999
2,Exacto,2,1,"(1.99723183391003, 0.998269896193771)",8.14166497048948e-6,"(-0.0941176470588179, 0.141176470588224)",0.169673001198293,0.0955882352941176
3,Exacto,3,1,"(1.99993079584775, 1.00006920415225)",4.78921468854311e-8,"(-0.00415224913494860, -0.00276816608996633)",0.00499038238818639,0.649999999999975
4,Exacto,4,1,"(1.99998371667006, 0.999989822918787)",2.81718511093664e-10,"(-0.000553633217994509, 0.000830449826991320)",0.000998076477639298,0.0955882352941192
5,Exacto,5,1,"(1.99999959291675, 1.00000040708325)",1.65716771344289e-12,"(-2.44249949115094e-5, -1.62833299413023e-5)",2.93551905189623e-5,0.649999999997818


<IPython.core.display.Latex object>

,Método,Iteración General,Iteración Interna,x_k,f(x_k),grad_f(x_k),||grad_f(x_k)||,lambda_k
0,Armijo,0,4,"(1.00000000000000, 1.50000000000000)",5.00000000000000,"(-16.0000000000000, 24.0000000000000)",28.8444102037119,0.0625
1,Armijo,1,4,"(1.37500000000000, 1.00000000000000)",0.781250000000000,"(-6.00000000000000, 8.00000000000000)",10.0000000000000,0.0625
2,Armijo,2,4,"(1.53125000000000, 0.843750000000000)",0.244140625000000,"(-2.50000000000000, 2.50000000000000)",3.53553390593274,0.0625
3,Armijo,3,4,"(1.60937500000000, 0.804687500000000)",0.152587890625000,"(-1.25000000000000, 0.625000000000000)",1.39754248593737,0.0625
4,Armijo,4,2,"(1.80468750000000, 0.804687500000000)",0.0762939453125000,"(-0.781250000000000, 0)",0.781250000000000,0.2500
5,Armijo,5,3,"(1.80468750000000, 0.902343750000000)",0.0381469726562500,"(0, -0.781250000000000)",0.781250000000000,0.1250
6,Armijo,6,2,"(1.90234375000000, 0.902343750000000)",0.0190734863281250,"(-0.390625000000000, 0)",0.390625000000000,0.2500
7,Armijo,7,3,"(1.90234375000000, 0.951171875000000)",0.00953674316406250,"(0, -0.390625000000000)",0.390625000000000,0.1250
8,Armijo,8,2,"(1.95117187500000, 0.951171875000000)",0.00476837158203125,"(-0.195312500000000, 0)",0.195312500000000,0.2500
9,Armijo,9,3,"(1.95117187500000, 0.975585937500000)",0.00238418579101563,"(0, -0.195312500000000)",0.195312500000000,0.1250
